In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account
from utils.utils_chain import get_current_tokens_for_address, get_token_details_for_address, get_all_token_nonces_details_for_account, dec_to_padded_hex
from utils.utils_tx import ESDTToken, NetworkProviders
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.dex_proxy_contract import DexProxyContract, DexProxyClaimRewardsEvent, DexProxyEnterFarmEvent, DexProxyExitFarmEvent, DexProxyRemoveLiquidityEvent, DexProxyAddLiquidityEvent

context = Context()

ADDRESS SETUP

In [ ]:
user = Account(pem_file="~/Documents/mx-sdk-py-exchange/wallets/C1.pem")
user.address = Address("erd1l4fgzgnt0tnrx5wdz6dqn5lyftcmkexaxm68p2qhed3tfchdgd5sm39a5z")
user2 = Account(pem_file="")
user2.address = Address("erd1fl59xh7utvkp662ncqzf4m9zasudwrmurekvm2xpt69mjvc784jq3cd3ut")
sc_owner = Account(pem_file="~/Documents/mx-sdk-py-exchange/wallets/C10.pem")
sc_owner.address = Address("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
user.sync_nonce(context.network_provider.proxy)
sc_owner.sync_nonce(context.network_provider.proxy)

CONTRACTS INIT

In [ ]:
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

CLAIM REWARDS

In [ ]:

txhash = fees_collector_contract.claim_rewards(user, context.network_provider.proxy)
used_api = context.network_provider.api

CLAIM ORIGINAL CALLER

In [ ]:

txhash = fees_collector_contract.claim_rewards_original_caller(user, context.network_provider.proxy, user)

CLAIM BOOSTED REWARDS

In [ ]:

txhash = fees_collector_contract.claim_boosted_rewards(user, context.network_provider.proxy)
used_api = context.network_provider.api

tokens_list = {}
tokens_list = get_current_tokens_for_address(user.address, context.network_provider.proxy)
if tokens_list.keys == 0:
    print("No tokens")
    raise
print(tokens_list.items())

# Get the transaction details
tx = used_api.get_transaction(txhash)
sender = tx.sender.bech32()

ops = tx.raw_response['data']
print(ops)


CHECK ENERGY AFTER CLAIM

In [ ]:
from utils import decoding_structures


current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)

get_total_energy_for_week_hex = fees_collector_contract.get_total_energy_for_week(context.network_provider.proxy , current_week)
print(get_total_energy_for_week_hex)
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.bech32(), context.network_provider.proxy ,current_week)
print(get_user_energy_hex)


user_energy_before, staking_stats_before = fees_collector_contract.get_all_stats(context.network_provider.proxy, user)
txhash = fees_collector_contract.claim_boosted_rewards(user, context.network_provider.proxy)
user_energy_after, staking_stats_after = fees_collector_contract.get_all_stats(context.network_provider.proxy, user)

if user_energy_before + staking_stats_after['total_energy_for_week'] != staking_stats_before:
    print(f"User energy is correct on farm") 


CHECK CLAIM

In [5]:
op_to_look_for = { # the Staked token transfered from staking contract to metastake contract
        "action": "MultiESDTNFTTransfer",
        "sender": user.address,
        "receiver": fees_collector_contract.address
    }
api = context.network_provider.api
op = fees_collector_contract.get_tx_op(txhash, op_to_look_for, api)
print(txhash)
print(op_to_look_for)
print(op)

NameError: name 'user' is not defined

allowExternalClaim

In [ ]:
txhash = fees_collector_contract.allow_external_claim(user, context.network_provider.proxy, True)
tx = used_api.get_transaction(txhash)
data = tx.raw_response['data']
print(data)

External Claim

In [ ]:
# opt_original_caller.address
txhash = fees_collector_contract.claim_for_user(user, user2.address, context.network_provider.proxy )

In [ ]:
#GET REWARDS FOR WEEK
txhash = fees_collector_contract.claim_for_user(user, user2, context.network_provider.proxy )

In [ ]:
#UPGRADE CONTRACT
from contracts.fees_collector_contract import FeesCollectorContract

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

# get_account_keys_online(router_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/pre_upgrade_router.json")
fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy,
                                 config.FEES_COLLECTOR_BYTECODE_PATH_V3, [], True)

============ VIEWS =================

In [ ]:
#GET CURRENT WEEK
current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
print(current_week)

In [ ]:
#GET USER ENERGY
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from utils.utils_chain import decode_merged_attributes

# data_fetcher = FeeCollectorContractDataFetcher(context.get_fee_collector_contract, context.network_provider.proxy.url)
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.bech32(), context.network_provider.proxy ,current_week)
print(get_user_energy_hex)

In [ ]:
#GET USER ENERGY FOR WEEK
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.to_bech32(), context.network_provider.proxy , current_week)
print(get_user_energy_hex)

In [ ]:
#GET TOTAL ENERGY FOR WEEK
get_total_energy_for_week_hex = fees_collector_contract.get_total_energy_for_week(context.network_provider.proxy , current_week-1)
print(get_total_energy_for_week_hex)

In [ ]:
#GET TOTAL REWARDS FOR WEEK
get_total_rewards_hex = fees_collector_contract.get_total_rewards_for_week(context.network_provider.proxy , current_week-1)
print(get_total_rewards_hex)

GET ALL STATS

In [ ]:

all_stats = fees_collector_contract.get_all_stats(context.network_provider.proxy, user, current_week)
print(all_stats)